# Visualization Design

By: *Tyler Biggs*

---

**Overview**

This notebook will go over the design of the vizualizations. It should also serve as a reference for future custom visualizations.

In [1]:
%load_ext autoreload
%autoreload 2
from pprint import pprint

In [2]:
import pandas as pd
import numpy as np
import re
import collections
import itertools
import functools

import bokeh as bk
import bokeh.io
import bokeh.models
import bokeh.layouts
import bokeh.plotting
bokeh.io.output_notebook()

# import holoviews as hv
# hv.extension('bokeh')

Loading BokehJS ...

In [6]:
# Path hack to allow imports from the parent directory.
import sys, os
sys.path.insert(0, os.path.abspath('../../'))

In [7]:
from isadream.isadream import modelUtils
from isadream.isadream import io
from isadream.isadream.models import nodal, elemental

---

## Dataflow

The data is transfered from the Drupal server as a `.json` file. Those files placed into a directory as the user requests them. That is, all the datasets that a user selects for any given visualziation are placed in a directory. These files are condensed into four dataframes per `.json` file.

In [8]:
# A demo json file is provided.
nmr_json_demo = modelUtils.SIPOS_DEMO
demo_base_path = modelUtils.BASE_PATH
print(nmr_json_demo, '\n', demo_base_path)

/home/tyler/git/isadream/isadream/demo_data/demo_json/sipos_2006_talanta_nmr_figs.json 
 /home/tyler/git/isadream/isadream/demo_data


---

In [9]:
demo_json = io.read_idream_json(nmr_json_demo)
node = io.parse_node_json(demo_json)

In [10]:
# node

In [11]:
# help(node)

In [14]:
demo_json = io.read_idream_json(nmr_json_demo)
node = io.parse_node_json(demo_json)

test_groups = (
    ('27 Al ppm', 'ppm', ("Al",)),
    ('Total Aluminate Concentration', 'Molar', ("Al", )),
    ('Counter Ion Concentration', 'Molar', ("Na+", "Li+", "Cs+", "K+", )),
    ('Counter Ion', 'Species', ("Na+", "Li+", "Cs+", "K+", )),
    ('Base Concentration', 'Molar', ("OH-", )),
)

node_assays = [io.build_node_data(a, test_groups) for a in node.assays]
node_assays
dfs = [pd.DataFrame(cds) for cds, _ in node_assays]
df = pd.concat(dfs)
df

,parent_node,assay_node,sample_node,27 Al ppm,Total Aluminate Concentration,Counter Ion Concentration,Counter Ion,Base Concentration
0,bd8cda09-18b1-40b8-a959-ab099f158bd3,ae910d8c-ede1-4efe-9498-9e4ceb8c2aa0,c7216c78-048b-4576-8108-fae52c6a99d2,79.90,2.93,2.93,K+,2.93
1,bd8cda09-18b1-40b8-a959-ab099f158bd3,ae910d8c-ede1-4efe-9498-9e4ceb8c2aa0,c7216c78-048b-4576-8108-fae52c6a99d2,79.84,4.92,4.92,K+,4.92
2,bd8cda09-18b1-40b8-a959-ab099f158bd3,ae910d8c-ede1-4efe-9498-9e4ceb8c2aa0,c7216c78-048b-4576-8108-fae52c6a99d2,79.72,6.85,6.85,K+,6.85
3,bd8cda09-18b1-40b8-a959-ab099f158bd3,ae910d8c-ede1-4efe-9498-9e4ceb8c2aa0,c7216c78-048b-4576-8108-fae52c6a99d2,79.66,9.13,9.13,K+,9.13
4,bd8cda09-18b1-40b8-a959-ab099f158bd3,ae910d8c-ede1-4efe-9498-9e4ceb8c2aa0,c7216c78-048b-4576-8108-fae52c6a99d2,79.66,10.71,10.71,K+,10.71
0,0a9c9a5c-5876-4270-8014-fd36023e48c9,e5c7b581-51e0-4f97-9d55-c8c73f4ce88d,876758b0-b448-49e1-9361-4c046964e9e8,79.92,0.66,0.66,Li+,0.66
1,0a9c9a5c-5876-4270-8014-fd36023e48c9,e5c7b581-51e0-4f97-9d55-c8c73f4ce88d,876758b0-b448-49e1-9361-4c046964e9e8,79.84,1.10,1.10,Li+,1.10
2,0a9c9a5c-5876-4270-8014-fd36023e48c9,e5c7b581-51e0-4f97-9d55-c8c73f4ce88d,876758b0-b448-49e1-9361-4c046964e9e8,79.66,1.64,1.64,Li+,1.64
3,0a9c9a5c-5876-4270-8014-fd36023e48c9,e5c7b581-51e0-4f97-9d55-c8c73f4ce88d,876758b0-b448-49e1-9361-4c046964e9e8,79.54,2.14,2.14,Li+,2.14
4,0a9c9a5c-5876-4270-8014-fd36023e48c9,e5c7b581-51e0-4f97-9d55-c8c73f4ce88d,876758b0-b448-49e1-9361-4c046964e9e8,79.36,2.59,2.59,Li+,2.59


In [21]:
metadata = [md for _, md in node_assays]
metadata
md = collections.ChainMap(*metadata)
# dict(metadata)

NameError: name 'node_assays' is not defined

In [25]:
md.keys()

KeysView(ChainMap(defaultdict(<class 'tuple'>, {'7bc0a784-2f8f-44ba-9041-a7e0d9812f89': ({'$id': 'https://lampdev02.pnl.gov/bigg006/idreamdrupal/', 'title': 'Sipos 2006, Talanta NMR Figures', 'filename': 'sipos_2006_talanta_nmr_figs.json', 'description': 'Extracted figures.', 'submissionDate': '2018-5-25', 'publicReleaseDate': '2006-03-13', 'experimentSubType': 'Al_NMR'}, [None]), 'e1988d5d-1cc3-4134-baa4-d164caf86b79': (NodeInfo(info={'dataFile': 'sipos_2006_talanta_fig_3_KOH.csv'}, name='NodeInfo00075'), [None]), '2a7e4c6b-864d-411a-8cff-a6b8e465e440': {'sampleName': 'Aluminate Solution'}, '18c201f8-cfd5-4d67-b82e-4d53d24dc1d8': {'sampleName': 'Aluminate Solution'}, '977d884f-a11b-4bb2-b4ef-3b25f966cc40': {'sampleName': 'Potassium Hydroxide'}, '728e2f7c-e1b0-4a80-b189-ec24c63a49bf': {'sampleName': 'Potassium Hydroxide'}, 'e1025e99-bafc-43f9-898a-bab9e40045fa': {'sampleName': 'Potassium Hydroxide'}}), defaultdict(<class 'tuple'>, {'9d992d88-f405-4481-94df-b67991dabf6b': ({'$id': 'http

---